In [31]:
from datetime import datetime
import pandas as pd

## Load data

In [32]:
cleaned_deduped_authors_filename = "data/MCK-CX_2022-10-09magazine_authors_newly_merged.csv"
newly_parsed_authors_filename = "data/magazine_authors_parsed.csv"
newly_merged_authors_filename = "data/magazine_authors-2023-03-25-merged.csv"

In [33]:
cleaned_deduped_authors = pd.read_csv(cleaned_deduped_authors_filename)

In [34]:
cleaned_deduped_authors.head()

,drupal_full_name,drupal_author_id,duplicate of ID,given_name,family_name,organization_name,meeting_name,Notes
0,Carl Abbott,1352,NaN,Carl,Abbott,NaN,NaN,NaN
1,Marge Abbott,2931,343.0,Margery Post,Abbott,NaN,NaN,NaN
2,Margery Post Abbott,3430,343.0,Margery Post,Abbott,NaN,NaN,NaN
3,Margery Abbott,3267,343.0,Margery Post,Abbott,NaN,NaN,NaN
4,Margery Post Abbott,343,NaN,Margery Post,Abbott,NaN,NaN,NaN


In [35]:
cleaned_deduped_authors.shape

(2774, 8)

In [36]:
newly_parsed_authors = pd.read_csv(newly_parsed_authors_filename)

In [37]:
newly_parsed_authors.head()

,given_name,family_name,drupal_full_name,drupal_author_id,civicrm_id
0,A. N.,Whitehead,A. N. Whitehead,1832,NaN
1,A.,S.,A. S.,1597,NaN
2,A. Sidney,Wright,A. Sidney Wright,1789,NaN
3,A. Stanley,Thompson,A. Stanley Thompson,2801,NaN
4,A.F.,Anderson,A.F. Anderson,2615,NaN


In [38]:
newly_parsed_authors.shape

(3106, 5)

## Merge new authors into existing spreadsheet

We will use the Drupal ID to skip existing rows and add only new authors to the merged spreadsheet.

In [39]:
# select newly parsed authors that are NOT in the cleaned deduped authors
new_authors = newly_parsed_authors[~newly_parsed_authors["drupal_author_id"].isin(cleaned_deduped_authors["drupal_author_id"])]

In [40]:
# Add a column to show the date new authors were added to the CSV
date_today = datetime.today().strftime('%Y-%m-%d')
new_authors.assign(date_added_to_list=date_today)

,given_name,family_name,drupal_full_name,drupal_author_id,civicrm_id,date_added_to_list
5,Abiel,Locke,Abiel Locke,5455,NaN,2023-03-25
10,African Great Lakes Initiative of Friends Peace,Teams,African Great Lakes Initiative of Friends Peac...,5367,NaN,2023-03-25
11,NaN,AFSC,AFSC,5542,1506.0,2023-03-25
12,AFSC Leadership,Team,AFSC Leadership Team,5373,NaN,2023-03-25
13,"AFSC Salem,",OR,"AFSC Salem, OR",5543,670.0,2023-03-25
...,...,...,...,...,...,...
3086,Woodbrooke Quaker Study,Centre,Woodbrooke Quaker Study Centre,5705,1178.0,2023-03-25
3087,Working Group on Right Relationship with,Animals,Working Group on Right Relationship with Animals,5473,NaN,2023-03-25
3088,World Beyond,War,World Beyond War,5451,NaN,2023-03-25
3090,Wyoming Friends,Meeting,Wyoming Friends Meeting,5706,45.0,2023-03-25


In [41]:
# Clear out any previous added date
cleaned_deduped_authors = cleaned_deduped_authors.assign(date_added_to_list=None)

In [42]:
# TODO: determine why date_added_to_list column isn't included in the newly merged authors for export
newly_merged_authors_for_export = pd.concat([cleaned_deduped_authors, new_authors])

In [43]:
newly_merged_authors_for_export

,drupal_full_name,drupal_author_id,duplicate of ID,given_name,family_name,organization_name,meeting_name,Notes,date_added_to_list,civicrm_id
0,Carl Abbott,1352,NaN,Carl,Abbott,NaN,NaN,NaN,None,NaN
1,Marge Abbott,2931,343.0,Margery Post,Abbott,NaN,NaN,NaN,None,NaN
2,Margery Post Abbott,3430,343.0,Margery Post,Abbott,NaN,NaN,NaN,None,NaN
3,Margery Abbott,3267,343.0,Margery Post,Abbott,NaN,NaN,NaN,None,NaN
4,Margery Post Abbott,343,NaN,Margery Post,Abbott,NaN,NaN,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...
3086,Woodbrooke Quaker Study Centre,5705,NaN,Woodbrooke Quaker Study,Centre,NaN,NaN,NaN,NaN,1178.0
3087,Working Group on Right Relationship with Animals,5473,NaN,Working Group on Right Relationship with,Animals,NaN,NaN,NaN,NaN,NaN
3088,World Beyond War,5451,NaN,World Beyond,War,NaN,NaN,NaN,NaN,NaN
3090,Wyoming Friends Meeting,5706,NaN,Wyoming Friends,Meeting,NaN,NaN,NaN,NaN,45.0


In [44]:
newly_merged_authors_for_export.sort_values(by="drupal_full_name", inplace=True)

In [45]:
# make sure the row counts match
assert new_authors.shape[0] + cleaned_deduped_authors.shape[0] == newly_merged_authors_for_export.shape[0]

## Export

In [46]:
newly_merged_authors_for_export.to_csv(newly_merged_authors_filename, index=False)